In [1]:
pip install Pyfhel

In [2]:
import numpy as np
from Pyfhel import Pyfhel

In [3]:
HE_client = Pyfhel(context_params={'scheme':'ckks', 'n':2**13, 'scale':2**30, 'qi_sizes':[30]*5})
print(f"1 - {HE_client}")
HE_client.keyGen()
print(f"2 - {HE_client}")
HE_client.relinKeyGen()
print(f"3 - {HE_client}")
HE_client.rotateKeyGen()
print(f"4 - {HE_client}")

1 - <ckks Pyfhel obj at 0x7b896a533be0, [pk:-, sk:-, rtk:-, rlk:-, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>
2 - <ckks Pyfhel obj at 0x7b896a533be0, [pk:Y, sk:Y, rtk:-, rlk:-, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>
3 - <ckks Pyfhel obj at 0x7b896a533be0, [pk:Y, sk:Y, rtk:-, rlk:Y, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>
4 - <ckks Pyfhel obj at 0x7b896a533be0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>


In [4]:
path="/content/drive/MyDrive/Colab Notebooks/Pipeline training & Predict trên dữ liệu mã hóa HE/data/client"

### <ckks Pyfhel obj at 0x7c3f2e2075f0, [pk:Y, sk:Y, rtk:Y, rlk:Y, contx(n=8192, t=0, sec=128, qi=[30, 30, 30, 30, 30], scale=1073741824.0, )]>

- **ckks Pyfhel obj** = Một instance của Pyfhel
- **0x7c3f2e2075f0** = địa chỉ trong bộ nhớ
- **pk** = public key Dùng để mã hóa plaintext thành ciphertext
- **sk** = secret key Dùng để giải mã
- **rtk** = relinearization key Dùng sau phép nhân ciphertext × ciphertext để giảm kích thước ciphertext
- **rtl** = rotation key Dùng để xoay/shift các slot trong vector CKKS (phải thiết nếu bạn muốn phép xoay vector, ví dụ để inner-product, sum across slots, v.v.)
- **contx**(...) — thông tin context (rất quan trọng) chứa các tham số hệ thống HE
- **n**=2**13=8192 = poly_modulus_degree (độ dài đa thức) càng lớn càng an toàn, 2^k
- **sec**=128 = 128 bit security
- '**qi_sizes**':[30]*5

Có 5 modulus [30] bit

CKKS khi nhân 2 ciphertext sẽ tạo ra nhiễu (noise)

Noise>modulus thì kết quả giải mã sai

→ CKKS rescale sau mỗi phép nhân (Mỗi lần rescale bạn mất 1 tầng modulus)

→ 1 rescale = giảm noise, giữ scale, mất 1 tầng modulus

scale=2^30, 1 phép nhân thì scale=2^60 → mất 1 tầng → còn 4 tầng

- **scale=2^30**

Độ chính xác (precision) của số thực được mã hóa

CKKS không lưu số thực float, mà lưu số nguyên int lớn

VD: Khi bạn mã hóa 3.1415, CKKS sẽ lưu 3.1xscale, nếu scale=2**30 thì CKKS sẽ lưu 3.1415x2^30=3 374 927 232

→ Scale càng lớn thì càng chính xác, ciphertext cũng lớn hơn → tốn modulus hơn


In [5]:
HE_client.save_context(path+"/context.txt")
HE_client.save_public_key(path+"/public_key.txt")
HE_client.save_relin_key(path+"/relin_key.txt")
HE_client.save_rotate_key(path+"/rotate_key.txt")

35482811

In [6]:
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:100, :]    # Lấy 100 mẫu đầu tiên
feature_names = np.array(iris.feature_names)
print(feature_names)
print(type(X[0]))
print(X[0])
y = X[:, 2]
print(type(y))
print(y[0])
X = np.delete(X, 2, axis=1)

['sepal length (cm)' 'sepal width (cm)' 'petal length (cm)'
 'petal width (cm)']
<class 'numpy.ndarray'>
[5.1 3.5 1.4 0.2]
<class 'numpy.ndarray'>
1.4


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
Enc_X = [HE_client.encryptFrac(x) for x in X_train]
Enc_y = HE_client.encryptFrac(y_train)
print(type(Enc_X[0]))
print(Enc_X[0])
print(type(Enc_y))
print(Enc_y)

<class 'Pyfhel.PyCtxt.PyCtxt'>
<Pyfhel Ciphertext at 0x7b8942573150, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>
<class 'Pyfhel.PyCtxt.PyCtxt'>
<Pyfhel Ciphertext at 0x7b8941e0df30, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>


In [9]:
import json

Enc_X_bytes = [e.to_bytes() for e in Enc_X]
Enc_y_bytes = Enc_y.to_bytes()

Enc_X_str = [estr.hex() for estr in Enc_X_bytes]
Enc_y_str = Enc_y_bytes.hex()

data = {
    "Enc_X": Enc_X_str,
    "Enc_y": Enc_y_str
}

with open(f"{path}/data.json", "w") as f:
    json.dump(data, f)

print("Save done")

Save done
